In [123]:
%matplotlib inline

import pandas as pd
import numpy as np
from numpy import linalg as LA
import matplotlib.pyplot as plt

## Reading Data

In [124]:
test_data = pd.read_csv('/home/asma/Documents/Intelligent Systems/Home Works/HW4_Datasets/Q1/Test_Data.csv', header=None)
test_labels = pd.read_csv('/home/asma/Documents/Intelligent Systems/Home Works/HW4_Datasets/Q1/Test_Labels.csv', header=None)
train_data = pd.read_csv('/home/asma/Documents/Intelligent Systems/Home Works/HW4_Datasets/Q1/Train_Data.csv', header=None)
train_labels = pd.read_csv('/home/asma/Documents/Intelligent Systems/Home Works/HW4_Datasets/Q1/Train_Labels.csv', header=None)

## Standardaize Data

In [125]:
def standardize(data):
    return (data - data.mean()) / data.std()

In [126]:
train_data = standardize(train_data)
test_data = standardize(test_data)

## Kmean++ from scratch

In [127]:
def K_means(k,data):
    
    conv = False
    data['labels'] = -1
    dis = pd.DataFrame(columns=[0, 1 ,2])
    dp = pd.DataFrame()
    centeroids = data.sample(n=1).drop('labels', axis =1).reset_index(drop=True)
    
    for i in range(0, k-1):
        cnt = centeroids.loc[i]
        dff = data.drop('labels', axis=1)-cnt.values.reshape((1, 13))
        dp = pd.concat((dff, dp), axis=1).max(axis=1).reset_index(drop=True)
        new = data.sample(n=1, weights = dp**2).drop('labels', axis =1)
        centeroids = pd.concat([centeroids, new])
        centeroids.reset_index(inplace=True, drop=True)
    
    while (conv == False):    
        for i, cnt in centeroids.iterrows():
            df = data.drop('labels', axis=1)-cnt.values.reshape((1, 13))
            dis[i] = LA.norm(df.values, axis=1)
            
        data['labels'] = dis.idxmin(axis=1)
        updates = data.groupby('labels').mean().reset_index(drop=True)
    
        if (updates.equals(centeroids)):
            conv = True
        else:
            centeroids = updates
        
    return data, centeroids


### Cost Function

In [128]:
def Intra_Cluster(cluster):
    df = cluster - cluster.mean()
    intra_distance = LA.norm(df, axis=1)
    return intra_distance.sum()

In [129]:
def cost_function(data):
    gdf= data.groupby('labels').apply(lambda group: Intra_Cluster(group))
    return gdf.sum()

### iterating clustring algorithm 100 time and choosing the one with minimum cost function

In [130]:
cluster , center= K_means(3, train_data)

for i in range(100):
    new, cnt = K_means(3, train_data)
    if ( cost_function(new) < cost_function(cluster)):
        cluster = new
        center = cnt

In [131]:
cost_function(cluster)

359.6766542570161

### prediction

In [132]:
def predict(test , centeroids):
    dis = pd.DataFrame(columns=[0, 1 ,2])

    for i, cnt in centeroids.iterrows():
            df =  test -cnt.values.reshape((1, 13))
            dis[i] = LA.norm(df.values, axis=1)
    
    test['labels'] = dis.idxmin(axis=1)

    return test['labels']

In [133]:
predicted_lables = predict(test_data, center)

### fixing labels

In [134]:
test_labels[0].value_counts()

3    14
2    10
1     9
Name: 0, dtype: int64

In [136]:
predicted_lables.value_counts()

1    13
2    10
0    10
Name: labels, dtype: int64

In [138]:
predicted_lables +=1

### Confusion Matrix

In [141]:
from sklearn.metrics import confusion_matrix
confusion_matrix(test_labels[0], predicted_lables)

array([[ 9,  0,  0],
       [ 1,  9,  0],
       [ 0,  4, 10]])

## SKlearn Algorithm

In [148]:
from sklearn.cluster import KMeans

new_train = train_data.to_numpy()
kmeans = KMeans(n_clusters=3)
sklearn_labels = kmeans.fit_predict(new_train)
kmean_centroid = kmeans.cluster_centers_

In [153]:
sklearn_labels

AttributeError: 'numpy.ndarray' object has no attribute 'value_counts'

In [154]:
confusion_matrix(test_labels[0], sklearn_lables)

NameError: name 'sklearn_lables' is not defined